In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [4]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [5]:
# fonction Drewsen wz/wx en fct de alphaRL
def alpha_dre(alpha):
    alpha1=sqrt(-2*(arcsin((1-alpha**(-2))**0.5)-alpha*(1-alpha**(-2))**0.5)/(arcsin((1-alpha**(-2))**0.5)-alpha**-1*(1-alpha**(-2))**0.5))
    alpha0=sqrt(-2*(arcsinh((alpha**(-2)-1)**0.5)-alpha*(alpha**(-2)-1)**0.5)/(arcsinh((alpha**(-2)-1)**0.5)-alpha**-1*(alpha**(-2)-1)**0.5))
    return alpha0,alpha1

def alpha_R(R,swx,kappa):
    e=1.602e-19
    m=40.078*1.66054e-27 # 40.078
    eps0 = 8.854187*1e-12
    rho=2*m*eps0*swx**2/e**2
    U0=m*d0**2*((swx**2*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[0])/(1+0.5*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[0]))/(2*kappa*e)
    U1=m*d0**2*((swx**2*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[1])/(1+0.5*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[1]))/(2*kappa*e)
    return U0,U1

In [6]:
def tick_function_q(my_q):
    V = my_q * 107.69 # 107.69123454143177
    return V

def tick_function_urf(my_Urf):
    my_q =  my_Urf / 107.69123454143178
    return my_q

def tick_function_a(my_a):
    V = my_a * 149.83128284025287
    return V

def tick_function_udc(Uend):
    aaz = Uend * 0.006674173650813496
    return aaz

In [140]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run '../../Functions/0.2/data_fct_Adrien_point_by_point.py'

In [8]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19          # e en conflit avec exponentielle de numpy !!
kb = 1.38064852*1e-23  # boltzman
m_Ca = 40.078*1.66054e-27 # Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [530]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try00
> Filename : Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 0


In [527]:
all_subdir

[]

In [504]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

IndexError: list index out of range

In [481]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg) 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 71
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09', 'Try10', 'Try11', 'Try12']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try01/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try02/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try03/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try04/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try05/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.5_S1.0RFG
00 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08

04 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF14/Try12/Temp_SimuType0_N01024_Vrf0061_Udc0.1000D+01V_D1.5_S1.0RFG
Point n° 5
05 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try00/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try01/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try02/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try03/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try04/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try05/Temp_SimuType0_N01024_Vrf0062_Udc0.1000D+01V_D1.5_S1.0RFG
05 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF15/Try06/Temp_SimuType0_N01024_Vrf0062_Udc0.1000

09 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF19/Try12/Temp_SimuType0_N01024_Vrf0055_Udc0.1000D+01V_D1.5_S1.0RFG
Point n° 10
10 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try01/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try02/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try03/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try04/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try05/Temp_SimuType0_N01024_Vrf0053_Udc0.2000D+01V_D1.5_S1.0RFG
10 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try06/Temp_SimuType0_N01024_Vrf0053_Udc0.200

14 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF14/Try12/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.5_S1.0RFG
Point n° 15
15 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try00/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try01/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try02/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try03/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try04/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try05/Temp_SimuType0_N01024_Vrf0062_Udc0.2000D+01V_D1.5_S1.0RFG
15 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF15/Try06/Temp_SimuType0_N01024_Vrf0062_Udc0.200

19 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF08/Try12/Temp_SimuType0_N01024_Vrf0053_Udc0.3000D+01V_D1.5_S1.0RFG
Point n° 20
20 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try00/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try01/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try02/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try03/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.5_S1.0RFG
20 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try06/Temp_SimuType0_N01024_Vrf0059_Udc0.300

...

49 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF11/Try12/Temp_SimuType0_N01024_Vrf0070_Udc0.6000D+01V_D1.5_S1.0RFG
Point n° 50
50 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try00/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try01/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try02/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try03/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try04/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try05/Temp_SimuType0_N01024_Vrf0061_Udc0.6000D+01V_D1.5_S1.0RFG
50 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF14/Try06/Temp_SimuType0_N01024_Vrf0061_Udc0.600

54 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC10_RF18/Try12/Temp_SimuType0_N01024_Vrf0071_Udc0.6000D+01V_D1.5_S1.0RFG
Point n° 55
55 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try01/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try02/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try03/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try04/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try05/Temp_SimuType0_N01024_Vrf0053_Udc0.7000D+01V_D1.5_S1.0RFG
55 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF08/Try06/Temp_SimuType0_N01024_Vrf0053_Udc0.700

59 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF14/Try12/Temp_SimuType0_N01024_Vrf0061_Udc0.7000D+01V_D1.5_S1.0RFG
Point n° 60
60 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try00/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try01/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try02/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try03/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try04/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try05/Temp_SimuType0_N01024_Vrf0062_Udc0.7000D+01V_D1.5_S1.0RFG
60 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC11_RF15/Try06/Temp_SimuType0_N01024_Vrf0062_Udc0.700

64 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC12_RF10/Try12/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.5_S1.0RFG
Point n° 65
65 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try01/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try02/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try03/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try04/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.5_S1.0RFG
65 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC13_RF10/Try06/Temp_SimuType0_N01024_Vrf0064_Udc0.333

69 - 12  >  /home/adrian/RemoteFS/Hobitton/20200131/DC17_RF10/Try12/Temp_SimuType0_N01024_Vrf0064_Udc0.5660D+01V_D1.5_S1.0RFG
Point n° 70
70 - 00  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 01  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try01/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 02  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try02/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 03  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try03/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 04  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try04/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 05  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.5_S1.0RFG
70 - 06  >  /home/adrian/RemoteFS/Hobitton/20200131/DC19_RF10/Try06/Temp_SimuType0_N01024_Vrf0064_Udc0.650

In [482]:
# check if each run gave birth to detection signal (SNR > some_threshold)
SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))
print(len(trig_detection))

[0.61538462 1.         1.         0.         1.         1.
 0.46153846 0.23076923 0.         1.         0.15384615 1.
 1.         0.         1.         1.         1.         1.
 0.         0.         0.23076923 1.         0.30769231 0.84615385
 0.84615385 1.         0.46153846 0.38461538 0.         0.
 0.92307692 0.84615385 0.15384615 0.30769231 1.         0.69230769
 0.         0.         0.07692308 0.46153846 0.38461538 0.53846154
 0.69230769 0.53846154 0.53846154 0.         0.         0.
 1.         0.69230769 0.92307692 0.92307692 1.         1.
 0.         0.         0.         0.53846154 0.69230769 0.
 0.07692308 1.         1.         0.         0.38461538 1.
 0.92307692 0.69230769 0.76923077 0.92307692 1.        ]
71


# Plots
Nuage d'ions (XYZ)
## stats en fonction de Udc (second order condition)
Delta F nuage Ca+
Delta E GMol
R/L
alpha

In [444]:
plot_point = 22
print(list(points_and_coord.keys())[plot_point])
plot_try = 1
plot_subdir = all_subdir[plot_point*13+plot_try%13]
print(plot_subdir)

DC05_RF14
/home/adrian/RemoteFS/Hobitton/20200131/DC05_RF14/Try01


In [144]:
# plot XYZ

print(plot_subdir)
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')

DC05_RF08
/home/adrian/Documents/Programmes/Fortran_Jofre/20200131/DC01_RF11/Try01


In [ ]:
# Plot T and PM

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_Inje_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou')
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [257]:
# Plot T with simulation time step
fname = 'only T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti)
ax.grid()

ax.set_xlabel('[ms]')
ax.set_ylabel('[cts/500ns]')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

In [456]:
# Plot T with custom time step

time_step = 1e-3 # s
custom_pts = len(ta)*(ta[1]-ta[0])/time_step
print(custom_pts)
somme_pts = int( round( time_step/(ta[1]-ta[0]) ) )
print(somme_pts)

custom_t = np.linspace(1,ceil(max(ta)/time_step),custom_pts+1)
custom_t_offset = max(custom_t) - max(ta)/time_step
custom_t = custom_t - custom_t_offset
custom_fluo = np.zeros( int(ceil(custom_pts)) )

for j in range( int(ceil(custom_pts))-1 ):
    temp = []
    print(j)
    print(somme_pts*j,somme_pts*(j+1)-1)
    for k in range(somme_pts*j,somme_pts*(j+1)):
        temp.append(ti[-k-1])
    temp = sum(temp)
    custom_fluo[-1-j] = temp/max(ti)/15

fname = 'only T custom T step'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti,zorder=10,label='500 ns')
# for j in range( int(ceil(custom_pts)) ):
#     ax.plot(ta[::-1][somme_pts*j:somme_pts*(j+1)]*1000,ti[::-1][somme_pts*j:somme_pts*(j+1)])
ax.scatter(custom_t[1:],custom_fluo[1:]/max(custom_fluo)*max(ti),s=200,marker='+',lw=5,edgecolor='black',zorder=100,label='1 ms')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax.grid()


ax.set_xlim(1.1,5.98)
ax.set_xlabel('[ms]')
ax.set_ylabel('[cts]')
ax.legend(title='Time step')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

5.9323951219512825
2030
0
0 2029
1
2030 4059
2
4060 6089
3
6090 8119
4
8120 10149


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


In [431]:
time_reverse = ta[::-1][0:2]
print(time_reverse)

[0.00593291 0.00593241]


In [223]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

(20, 21)

In [483]:
ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]

fname = '3D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111, projection='3d')

# fig1 = plt.figure(num=fname,clear=True)
# ##### PLOT DFluo et Proba détection
# ax1 = fig1.add_subplot(111)
# ax2 = ax1.twinx()
# clf()

l = 0
plot_Udc = []
plot_Urf = []
Z = []
for i,j in enumerate(points_and_coord):
    plot_Udc.append(Udc[int(points_and_coord[j][0])])
    plot_Urf.append(Urf[int(points_and_coord[j][1])])
    Z.append(Gmol_data[2][i])
            # ax.set_aspect('equal')
Z = np.array(Z)
print(shape(plot_Udc),shape(plot_Urf),shape(Z))

for k in range(13):
    ax.scatter(xs=plot_Udc,ys=plot_Urf,zs=Z[:,k],color='xkcd:azul',marker='o',s=10)
#             ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

#             ax.view_init(elev=18, azim=55)
#             ax.set_xlim(-3e-1,3e-1)
#             ax.set_ylim(-3e-1,3e-1)
#             ax.set_zlim(-3e-1,3e-1)

#             ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\n$U_{DC}$ [V]')
ax.set_ylabel('\n$U_{RF}$ [V]')
ax.set_zlabel('\n SNR ')

#         ax.plot(np.ones(100)*Udc[dc], np.ones(100)*Urf[rf], zs=linspace(0,0.8,100),color='black',ls=':',lw=1)

# X = [Udc for hhh in range(elem_1+1)]
# Y = [Urf for smeuiofh in range(elem_2-1)]
# # X, Y = np.meshgrid(X, Y)
# Z = asarray(transpose(mean_fluo))
# Z = np.meshgrid(Z)

# # surf = ax.plot_wireframe(transpose(X), Y, Z)
# surf = ax.plot_surface(transpose(X), Y, Z, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='black',alpha = 0.35)
# #         ax.errorbar(Udc[dc],mean_fluo[],yerr=prob_detect_xerr,xerr=None,
# #              elinewidth=2.5,capsize=None,color='xkcd:yellow orange',
# #              fmt=':+',ms=12,mew=3.5,mfc='None',mec='xkcd:yellow orange')
# cbar = plt.colorbar(surf)
# cbar.ax.set_ylabel(r'$\Delta F / F$ moyen ('+str(elem_0)+' essais)', rotation=90)


fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
# subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

(71,) (71,) (71, 13)


## 2D plot

In [495]:
for i,j in enumerate(points_and_coord):
    print(j)
    print(points_and_coord[j][0],points_and_coord[j][1])

DC01_RF08
01 08
DC01_RF09
01 09
DC01_RF10
01 10
DC01_RF11
01 11
DC01_RF14
01 14
DC01_RF15
01 15
DC01_RF16
01 16
DC01_RF17
01 17
DC01_RF18
01 18
DC01_RF19
01 19
DC03_RF08
03 08
DC03_RF09
03 09
DC03_RF10
03 10
DC03_RF11
03 11
DC03_RF14
03 14
DC03_RF15
03 15
DC03_RF16
03 16
DC03_RF17
03 17
DC03_RF18
03 18
DC05_RF08
05 08
DC05_RF09
05 09
DC05_RF10
05 10
DC05_RF11
05 11
DC05_RF14
05 14
DC05_RF15
05 15
DC05_RF16
05 16
DC05_RF17
05 17
DC05_RF18
05 18
DC07_RF08
07 08
DC07_RF09
07 09
DC07_RF10
07 10
DC07_RF11
07 11
DC07_RF14
07 14
DC07_RF15
07 15
DC07_RF16
07 16
DC07_RF17
07 17
DC07_RF18
07 18
DC09_RF08
09 08
DC09_RF09
09 09
DC09_RF10
09 10
DC09_RF11
09 11
DC09_RF14
09 14
DC09_RF15
09 15
DC09_RF16
09 16
DC09_RF17
09 17
DC09_RF18
09 18
DC10_RF08
10 08
DC10_RF09
10 09
DC10_RF10
10 10
DC10_RF11
10 11
DC10_RF14
10 14
DC10_RF15
10 15
DC10_RF16
10 16
DC10_RF17
10 17
DC10_RF18
10 18
DC11_RF08
11 08
DC11_RF09
11 09
DC11_RF10
11 10
DC11_RF11
11 11
DC11_RF14
11 14
DC11_RF15
11 15
DC11_RF16
11 16
DC11_RF1

In [240]:
# plot delta E Gmol absolu
SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shapevar)
rtoLratio = zeros(shapevar)

fname = 'Fluo_stat_all'
fig1 = plt.figure(num=fname,clear=True)
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(gs[0])
ax2 = ax1.twinx()
# clf()

l = 0
for i,j in enumerate(points_and_coord):
    plot_Udc = Udc[int(points_and_coord[j][0])]
    plot_Urf = Udc[int(points_and_coord[j][1])]
    # a,q attendu
    az.append(8*mkappa*(C_e/m_Ca)*plot_Udc/(d0**2*Omega**2))
    qx.append(-4*(C_e/m_Ca)*plot_Urf/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
#     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*C_e*plot_Udc/(m_Ca*d0**2))/(2*pi))
        
    
    for k in range(elem_0): # try
        if PMvar[k][my_order[j]] > 0.05:
            sigdetect[j] +=1
        alpha_dre[k][j] = wz[j]**2/wx[j][plot_Urf]**2
        rtoLratio[k][j] = dim_nu[k,j,0] / dim_nu[k,j,2]
           
        ax1.plot(Udc[j],PMvar[k][my_order[j]],'o',color='xkcd:azul',markerfacecolor='none')
    
    l+=1
    
for k in range(0,len(Udc_no_repeat),3):
        ax1.annotate('a={:1.1e}\nwz={:3.1f}kHz'.format(az[k],wz[k]/1000), xy=(Udc[k],0.73), xycoords='data',
        size=8, ha='center', va='top', color='xkcd:azul',
        bbox=dict(boxstyle='round', fc='none',edgecolor='none')) 

mean_detect = [mean_fluo[k]/max(mean_fluo) for k in range(len(mean_fluo))]
prob_detect = sigdetect/elem_0
prob_detect_xerr = sqrt(prob_detect*(1-prob_detect))/sqrt(elem_0)
        
ax2.errorbar(Udc_no_repeat,prob_detect,yerr=prob_detect_xerr,xerr=None,
             elinewidth=1,capsize=None,color='xkcd:yellow orange',
             fmt=':+',ms='8',mew=2,mfc='None',mec='xkcd:yellow orange')
annotate('qx={:1.1e}\nUrf={:3.1f}V\nwx={:3.1f}kHz'.format(qx[0],Urf_no_repeat,wx[0]/1000), xy=(2,0.25), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul'))

ax1.set_ylabel('$\Delta F / F$', color='xkcd:azul')
ax2.set_ylabel('$P_{Detect}$'+' ('+str(elem_0)+' runs)', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')


ax1.set_ylim(-0.05,1.1)
ax2.set_ylim(-0.05,1.1)
ax1.grid()

##### Plot DEnergie et moyenne de DEnergie
renorm = 0.5*m_GM/e
ax3 = fig1.add_subplot(gs[2],sharex=ax1)
ax4 = ax3.twinx()
# clf()
for j in range(len(Udc)):
    for k in range(elem_0):        
        ax3.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color='xkcd:azul',markerfacecolor='none')

mean_deltaEc = [mean(deltaEc[:,j])*renorm*1000 for j in my_order]
        
ax4.plot(Udc,mean_deltaEc,linestyle='--',marker='o',color='xkcd:yellow orange')
ax4.annotate('$E_{GMol}$=50 eV', xy=(1,5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax3.set_xlabel('$U_{DC}$')
ax3.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax4.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax3.tick_params(axis='y', labelcolor='xkcd:azul')
ax4.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax3.set_ylim(-0.095*1000,7)
ax4.set_ylim(-0.095*1000,7)
ax3.grid()

ax3.set_xlim(-0.1,8.1)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

##### Plot R/L et alpha
ax5 = fig1.add_subplot(gs[1],sharex=ax1)
ax6 = fig1.add_subplot(gs[3],sharex=ax5)

mean_rtoLratio = [mean(rtoLratio[:,j]) for j in my_order]
for j in range(elem_2):
    for k in range(elem_0):        
        ax5.plot(Udc[j],rtoLratio[k][my_order[j]],'o',color='xkcd:azul',markerfacecolor='none')
    ax6.plot(Udc[j],alpha_dre[0][j],'o',color='xkcd:azul')
    
ax5.plot(Udc,mean_rtoLratio,linestyle='--',marker='o',color='xkcd:yellow orange',markerfacecolor='none')

ax6.set_xlabel('$U_{DC}$')
ax5.set_ylabel('R/L')
ax6.set_ylabel(r'$\alpha$')
# ax5.tick_params(axis='y', labelcolor='xkcd:azul')
# ax6.tick_params(axis='y', labelcolor='xkcd:yellow orange')

# ax6.set_xlim(-0.1,8.1)
# ax5.set_ylim(-1.65,0.35)
# ax6.set_ylim(-1.65,0.35)
ax5.grid()
ax6.grid()

ax5.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
plt.setp(ax5.get_xticklabels(), visible=False)

tight_layout()

fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()
subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in sqrt


NameError: name 'elem_0' is not defined

In [491]:
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]

In [494]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(mean(trig_detection[g]))
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)

im = ax.scatter(x,y,
        marker='o',s=100,edgecolor='black',
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()
title(all_subdir[0][:-15],fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname,dpi=300)

In [1212]:
fname = 'art'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)

im = ax.contourf([1,2,3],[1,2,3],[[1,2,3],[4,5,6],[7,8,9]]) # [[nan,2,nan],[4,5,6],[nan,8,nan]]
# for rf in range(1,4):
#     for dd in range(1,4):
#         ax.plot(rf,dd,marker='+',ls='',markersize=10,mew=3,color = 'xkcd:yellow orange')

# ax.set_zlabel(r'$\Delta F / F$')
# im2 = ax.contour(Udc, Urf, transpose(Z), colors='k')
ax.set_xlim(0.95,3.05)
ax.set_ylim(0.95,3.05)
fig.set_size_inches(11,8)
fig.tight_layout()
# savefig(fname+str(40),dpi=300)

# retrieve additionnal data

In [163]:
# comparing amplitude RF mvmnt to Thermal mvmnt

to_calc = 4
T = 6e-4

Er = 2*Urf*mean(dim_nu[:,my_order[to_calc],0])/r0

RF_amp = C_e*Er / (m_Ca*Omega**2)
Th_amp = sqrt(kb*T**2/ (m_Ca*wx[my_order[to_calc]]**2) )

print(Th_amp, RF_amp)

IndexError: list index out of range

# Delta E analysis

Histogramme DeltaE GMol

In [82]:
# histogram for Delta energy
# two families : detection and non detection
trig_threshold = 0.6
my_trigger = [ [ deltaEc[k][my_order[j]]*renorm*1000 for k in range(elem_0) if PMvar[k][my_order[j]]>trig_threshold] for j in range(elem_2) ]
# my_DE = [ [deltaEc[k,my_selection[l]]*renorm*1000 for k in range(elem_0)] for l in range(elem_2) ]
y_hist_detec = concatenate(( [my_trigger[my_order[j]] for j in range(elem_2)] ))

my_nontrigger = [ [ deltaEc[k][my_order[j]]*renorm*1000 for k in range(elem_0) if PMvar[k][my_order[j]]<trig_threshold] for j in range(elem_2) ]
# my_DE = [ [deltaEc[k,my_selection[l]]*renorm*1000 for k in range(elem_0)] for l in range(elem_2) ]
y_hist_nondetec = concatenate(( [my_nontrigger[my_order[j]] for j in range(elem_2)] ))


fig = plt.figure(num='Histogram Delta energy GMol', clear=True)
# plt.suptitle('Histogram $\Delta E$')
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex=ax1)
 # PMvar[k][my_selection]
my_bins = arange(-120,0,2.5)
ax1.set_title(r'Detection $\Delta F/F>0.6$',color='xkcd:leaf green')
ax1.hist(y_hist_detec,bins=my_bins ,color='xkcd:leaf green', edgecolor='black', linewidth=1.2)
ax2.set_xlabel(r'$\Delta E$ [meV]')
ax1.set_ylim(0,35)
ax1.grid()
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))
ax1.annotate(f'{len(y_hist_detec)} cases', xy=(-123,33), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:leaf green',
    bbox=dict(boxstyle='round', fc='white')) 

ax2.set_title(r'No detection $\Delta F/F<0.6$',color='xkcd:rouge')
ax2.hist(y_hist_nondetec,bins=my_bins ,color='xkcd:rouge', edgecolor='black', linewidth=1.2)
ax2.set_ylim(0,35)
ax2.grid()
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=4))
ax2.annotate(f'{len(y_hist_nondetec)} cases', xy=(-123,33), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:rouge',
    bbox=dict(boxstyle='round', fc='white')) 

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.22,top=0.94)

# savefig('hist_simu.png', dpi=300)

NameError: name 'elem_2' is not defined

In [1071]:
# plot DE vs Udc but with red green colors

# my_order = [0,1,2,3,4,5,6,13,7,8,9,14,15,10,16,11,12] # 13,7,8,9,14,15,10,16,11,12

# Udc = [0.5,1,1.5,2,2.5,3,3.5,3.75,4,4.5,5,5.3,5.7,6,6.5,7,8]
# Urf = 64.61

mean_fluo = [mean(PMvar[:,j]) for j in my_order]

fig1 = plt.figure(num='Fluo var only_DE_colors',clear=True)

##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(111)
ax2 = ax1.twinx()

left, bottom, width, height = [0.4145, 0.12, 0.45, 0.5] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

my_color = ['xkcd:rouge', 'xkcd:leaf green']
l = 0
for j in range(len(Udc)):          
    # a,q attendu
    az.append(8*mkappa*(e/m)*Udc[j]/(d0**2*Omega**2))
    qx.append(-4*(e/m)*Urf/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
#     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*e*Udc[j]/(m*d0**2))/(2*pi))
    
    for k in range(elem_0):
        trig = 0
        if PMvar[k][my_order[j]] > 0.6:
            sigdetect[j] +=1
            trig = 1
        alpha_dre[k][j] = wz[j]**2/wx[j]**2
        rtoLratio[k][j] = dim_nu[k,j,0] / dim_nu[k,j,2]
        ax1.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
        axins.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
    
    l+=1


mean_deltaEc = [mean(deltaEc[:,j])*renorm*1000 for j in my_order]
        
ax2.plot(Udc,mean_deltaEc,linestyle=':',marker='>',color='xkcd:yellow orange',markerfacecolor='none')
ax2.annotate('$E_{GMol}$=50 eV', xy=(1,-5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax1.set_xlabel('$U_{DC}$')
ax1.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax2.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax1.set_ylim(-0.10*1000,1)
ax2.set_ylim(-0.10*1000,1)
ax1.grid()

ax2.set_xlim(-0.1,8.5)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))

x1, x2, y1, y2 = 3.25, 8.25, -21, -2.5 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.grid()

mark_inset(ax1, axins, loc1=2, loc2=1, fc="none", ec="0.4")
# axins.invert_yaxis()
plt.setp(axins.get_xticklabels(), visible=False)
plt.setp(axins.get_yticklabels(), fontsize=10)


fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


# Trajectory
Trajectoire GMol

In [729]:
'{}/trj{}'.format(fileload[0][0],data4[0][0][-60:].strip('.dat'))
trj_len = shape(load_trj(str_load))[1]
t = zeros((elem_0,elem_2,trj_len))
r_x = zeros((elem_0,elem_2,trj_len))
r_y = zeros((elem_0,elem_2,trj_len))
r_z = zeros((elem_0,elem_2,trj_len))

In [730]:
# k = 3
# j = 3

for j in range(elem_2):     # Udc
    print('j - k')
    for k in range(elem_0): # try
        str_load = '{}/trj{}'.format(fileload[k][j],data4[k][j][-60:].strip('.dat'))
        print('>',j,'-',k,str_load[69:])
        t[k,j], r_x[k,j], r_y[k,j], r_z[k,j], v_x, v_y, v_z, a_x, a_y, a_z = load_trj(str_load)

j - k
> 0 - 0 try0/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 1 try1/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 2 try2/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 3 try3/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 4 try4/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 5 try5/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 6 try6/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 7 try7/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 8 try8/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 9 try9/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 10 try10/Vrf10/Udc0/trj_SimuType4_01_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG50eV
> 0 - 11 try11/Vrf10/Udc

> 3 - 21 try21/Vrf10/Udc3/trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV
> 3 - 22 try22/Vrf10/Udc3/trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV
> 3 - 23 try23/Vrf10/Udc3/trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV
> 3 - 24 try24/Vrf10/Udc3/trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV
j - k
> 4 - 0 try0/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 1 try1/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 2 try2/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 3 try3/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 4 try4/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 5 try5/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 6 try6/Vrf10/Udc4/trj_SimuType4_01_N01024_Vrf0064_Udc0.2500D+01V_D1.0_S1.0RFG50eV
> 4 - 7 try7/Vrf10

> 11 - 3 try3/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 4 try4/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 5 try5/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 6 try6/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 7 try7/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 8 try8/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 9 try9/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 10 try10/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 11 try11/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 12 try12/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 11 - 13 try13/Vrf10/Udc11/trj_SimuType4_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG50eV
> 

> 14 - 19 try19/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
> 14 - 20 try20/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
> 14 - 21 try21/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
> 14 - 22 try22/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
> 14 - 23 try23/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
> 14 - 24 try24/Vrf10/Udc14/trj_SimuType4_01_N01024_Vrf0064_Udc0.5300D+01V_D1.0_S1.0RFG50eV
j - k
> 15 - 0 try0/Vrf10/Udc15/trj_SimuType4_01_N01024_Vrf0064_Udc0.5700D+01V_D1.0_S1.0RFG50eV
> 15 - 1 try1/Vrf10/Udc15/trj_SimuType4_01_N01024_Vrf0064_Udc0.5700D+01V_D1.0_S1.0RFG50eV
> 15 - 2 try2/Vrf10/Udc15/trj_SimuType4_01_N01024_Vrf0064_Udc0.5700D+01V_D1.0_S1.0RFG50eV
> 15 - 3 try3/Vrf10/Udc15/trj_SimuType4_01_N01024_Vrf0064_Udc0.5700D+01V_D1.0_S1.0RFG50eV
> 15 - 4 try4/Vrf10/Udc15/trj_SimuType4_01_N01024_Vrf0064_Udc0.5700D+01V_D1.0_S1.0

In [1497]:
# str_load = '{}/trj{}'.format(fileload[k][j],data4[k][j][-60:].strip('.dat'))
# print(str_load)
# t,r_x, r_y, r_z, v_x, v_y, v_z, a_x, a_y, a_z = load_trj(str_load)
cm = pylab.get_cmap('autumn')
# color=cm(1.*k/elem_0)
my_colors=['xkcd:reddish', 'xkcd:avocado','xkcd:dusty orange', 'xkcd:soft blue']
my_ls = '-'
my_lw = 1
fname = '2D_GMol_trajectory'
fig = plt.figure(num=fname,clear=True)
gs = gridspec.GridSpec(2, 2, width_ratios=[8, 6]) 
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[2],sharex=ax1)
ax3 = fig.add_subplot(gs[1],sharey=ax1)
ax4 = fig.add_subplot(gs[3], projection='3d')

trj_quad = [ [[] for l in range(4)] for k in range(elem_0) ] # k sortant dans chaque quadrant, pour chaque j
for j in range(elem_2):
    for k in range(elem_0):
        if r_x[k,j,-1]>0 and r_y[k,j,-1]>0:
            trj_quad[j][0].append(k)
        if r_x[k,j,-1]<0 and r_y[k,j,-1]>0:
            trj_quad[j][1].append(k)
        elif r_x[k,j,-1]<0 and r_y[k,j,-1]<0:
            trj_quad[j][2].append(k)
        elif r_x[k,j,-1]>0 and r_y[k,j,-1]<0:
            trj_quad[j][3].append(k)

# k = 3
j = 3
            
for k in trj_quad[j][0]:
    ax1.plot(r_z[k,j]*1e3,r_y[k,j]*1e9,color=my_colors[0],ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax1.plot(r_z[k,j]*1e3,r_y[k,j]*1e9,color=my_colors[1],ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax1.plot(r_z[k,j]*1e3,r_y[k,j]*1e9,color=my_colors[2],ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax1.plot(r_z[k,j]*1e3,r_y[k,j]*1e9,color=my_colors[3],ls=my_ls,lw=my_lw)
    
ax1.vlines(dim_nu[k,j,2],-100,100,linestyles='dotted',color='grey')
ax1.vlines(-dim_nu[k,j,2],-100,100,linestyles='dotted',color='grey')

ax1.set_xlabel('z [mm]')
ax1.set_ylabel('y [nm]')
ax1.set_xlim(-0.5,1.55)
ax1.xaxis.set_label_position('top') 
ax1.xaxis.tick_top()


ax1.grid()

for k in trj_quad[j][0]:
    ax2.plot(r_z[k,j]*1e3,r_x[k,j]*1e9,color=my_colors[0],ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax2.plot(r_z[k,j]*1e3,r_x[k,j]*1e9,color=my_colors[1],ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax2.plot(r_z[k,j]*1e3,r_x[k,j]*1e9,color=my_colors[2],ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax2.plot(r_z[k,j]*1e3,r_x[k,j]*1e9,color=my_colors[3],ls=my_ls,lw=my_lw)

ax2.vlines(dim_nu[k,j,2],-100,100,linestyles='dotted',color='grey')
ax2.vlines(-dim_nu[k,j,2],-100,100,linestyles='dotted',color='grey')
    
# for k in range(elem_0):
#     ax2.plot(r_z[k,j]*1e3,r_x[k,j]*1e9,color=cm(1.*k/elem_0))
# ax2.set_xlabel('z [mm]')
ax2.set_ylabel('x [nm]')
ax2.set_xlim(-0.5,1.55)
plt.setp(ax2.get_xticklabels(), visible=False)
ax2.grid()

for k in trj_quad[j][0]:
    ax3.plot(r_x[k,j]*1e9,r_y[k,j]*1e9,color=my_colors[0],ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax3.plot(r_x[k,j]*1e9,r_y[k,j]*1e9,color=my_colors[1],ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax3.plot(r_x[k,j]*1e9,r_y[k,j]*1e9,color=my_colors[2],ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax3.plot(r_x[k,j]*1e9,r_y[k,j]*1e9,color=my_colors[3],ls=my_ls,lw=my_lw)

# for k in range(elem_0):
#     ax3.plot(r_x[k,j]*1e9,r_y[k,j]*1e9,color=cm(1.*k/elem_0))
ax3.axis('equal')
ax3.set_xlabel('x [nm]')
# ax3.set_ylabel('y [nm]')
plt.setp(ax3.get_yticklabels(), visible=False)
ax3.xaxis.set_label_position('top') 
ax3.xaxis.tick_top()
ax3.grid()

for k in trj_quad[j][0]:
    ax4.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[0],lw=my_lw)
for k in trj_quad[j][1]:
    ax4.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[1],lw=my_lw)
for k in trj_quad[j][2]:
    ax4.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[2],lw=my_lw)
for k in trj_quad[j][3]:
    ax4.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[3],lw=my_lw)

# for k in range(elem_0):
#     ax4.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=cm(1.*k/elem_0))
ax4.plot(linspace(0,-125,1000),linspace(0,0,1000),zs=linspace(0,0,1000),color='grey',ls='--')
ax4.plot(linspace(0,0,1000),linspace(-1.5,1.5,1000),zs=linspace(0,0,1000),color='grey',ls='--')
ax4.plot(linspace(0,0,1000),linspace(0,0,1000),zs=linspace(-0,10,1000),color='grey',ls='--')
ax4.view_init(elev=35, azim=35)
ax4.set_xlabel('x')
ax4.set_ylabel('z')
ax4.set_zlabel('y')

ax1.annotate(f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', xy=(0.02,0.968), xycoords='axes fraction',
                size=11, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='xkcd:white'))

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(wspace=0.03,hspace=0.01)

savefig(fname,dpi=300)


In [1486]:
# Plot 3D avec trajectoires projetées
# le plot de la folie

def cc(arg):
    '''
    Shorthand to convert 'named' colors to rgba format at 60% opacity.
    '''
    return mcolors.to_rgba(arg, alpha=0.55)

j = 6
fname = '3D_GMol_trajectory'
fig = plt.figure(fname)
ax = fig.add_subplot(111, projection='3d')
# title('coucou')

ax.plot(linspace(-500,125,1000),linspace(0,0,1000),zs=linspace(0,0,1000),color='grey',ls='--')
ax.plot(linspace(0,0,1000),linspace(-2,2,1000),zs=linspace(0,0,1000),color='grey',ls='--')
ax.plot(linspace(0,0,1000),linspace(0,0,1000),zs=linspace(-500,100,1000),color='grey',ls='--')
    
xpos = [-500 for k in range(62)] # position plot zy
ypos = [-500 for k in range(62)] # position plot zx
zpos = [-2 for k in range(62)]   # position plot xy

# plot zy
for k in trj_quad[j][0]:
    ax.plot(xpos,r_z[k,j]*1e3,r_y[k,j]*1e9,color=cc(my_colors[0]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax.plot(xpos,r_z[k,j]*1e3,r_y[k,j]*1e9,color=cc(my_colors[1]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax.plot(xpos,r_z[k,j]*1e3,r_y[k,j]*1e9,color=cc(my_colors[2]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax.plot(xpos,r_z[k,j]*1e3,r_y[k,j]*1e9,color=cc(my_colors[3]),ls=my_ls,lw=my_lw)
    
# plot zx    
for k in trj_quad[j][0]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1e3,ypos,color=cc(my_colors[0]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1e3,ypos,color=cc(my_colors[1]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1e3,ypos,color=cc(my_colors[2]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1e3,ypos,color=cc(my_colors[3]),ls=my_ls,lw=my_lw)
    
# plot xy
for k in trj_quad[j][0]:
    ax.plot(r_x[k,j]*1e9,zpos,r_y[k,j]*1e9,color=cc(my_colors[0]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][1]:
    ax.plot(r_x[k,j]*1e9,zpos,r_y[k,j]*1e9,color=cc(my_colors[1]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][2]:
    ax.plot(r_x[k,j]*1e9,zpos,r_y[k,j]*1e9,color=cc(my_colors[2]),ls=my_ls,lw=my_lw)
for k in trj_quad[j][3]:
    ax.plot(r_x[k,j]*1e9,zpos,r_y[k,j]*1e9,color=cc(my_colors[3]),ls=my_ls,lw=my_lw)
    
# ax.plot(xs=r_x[k,j]*1000,ys=r_z[k,j]*1000,zs=r_y[k,j]*1000,color='xkcd:rouge')
# ax.plot(xs=r_LC[:][0],ys=r_LC[:][2],zs=r_LC[:][1],marker='o',ms=0.45,ls='')

# PLOT 3D
for k in trj_quad[j][0]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[0],lw=my_lw)
for k in trj_quad[j][1]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[1],lw=my_lw)
for k in trj_quad[j][2]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[2],lw=my_lw)
for k in trj_quad[j][3]:
    ax.plot(r_x[k,j]*1e9,r_z[k,j]*1000,zs=r_y[k,j]*1e9,color=my_colors[3],lw=my_lw)

ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
    
# ax.annotate(f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', xy=(400,-2), xycoords='axes fraction',
#                 size=11, ha='left', va='top',
#                 bbox=dict(boxstyle='round', fc='xkcd:white'))

    
ax.set_xlim(-500,500)
ax.set_ylim(-2,1.55)
ax.set_zlim(-500,500)
ax.view_init(elev=18, azim=55)

ax.set_xlabel('x [nm]')
ax.set_ylabel('z [mm]')
ax.set_zlabel('y [nm]')

# ax.zaxis.set_tick_params(labelsize=10,coucou=8)

# ax.set_zticklabels([0,1,2,3],rotation=-15,
#                    verticalalignment='baseline',
#                    horizontalalignment='left')

# ax.set_xlim(-0.25,0.25)
# ax.set_ylim(-0.25, 0.25)
# ax.set_zlim(-0.25, 0.25)
subplots_adjust(top=1, bottom = 0)
# fig.tight_layout()
fig.set_size_inches(11.69, 8.27)
savefig(fname,dpi=300)

In [1498]:
k = 13
j = 6

adress = work_rep[:myslashpos[slashcond]]+str(cond_zero_name)+str(k)+str(cond_one_name)+str(key1)+str(cond_two_name)+str(key2[j])
fileload[k].append(adress)

# get only .dat files in each simulation directory
onlyfiles = [f for f in listdir(fileload[k][j]) if isfile(join(fileload[k][j], f)) and not "xva" in f and ".dat" in f]

# load cloud size before injection
my_file = '{}/xva{}'.format(fileload[k][j],sort(onlyfiles)[0].strip('.dat')[4:])
r_LC,v_LC,a_LC = load_xyz_init_bin_DP(my_file)        

# filter lost ions
x_LC_clip = [r_LC[0,x] for x in range(len(r_LC[0,:])) if abs(r_LC[0,x]) <6e-2]
y_LC_clip = [r_LC[1,x] for x in range(len(r_LC[1,:])) if abs(r_LC[1,x]) <6e-2]
z_LC_clip = [r_LC[2,x] for x in range(len(r_LC[2,:])) if abs(r_LC[2,x]) <1e-0]
print(max(z_LC_clip))
#         r_LC_clip[k][j][:] = [x_LC_clip,y_LC_clip,z_LC_clip]
r_LC_clip = [x_LC_clip,y_LC_clip,z_LC_clip]


fname = '3D_GMol_trajectory_with_cloud'
fig = plt.figure(fname,clear='True')
ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')
ax.plot(xs=r_x[k,j]*1000,ys=r_z[k,j]*1000,zs=r_y[k,j]*1000,color='xkcd:vermillion')
ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

ax.view_init(elev=18, azim=55)
ax.set_xlim(-3e-1,3e-1)
ax.set_ylim(-3e-1,3e-1)
ax.set_zlim(-3e-1,3e-1)

ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\nx [mm]')
ax.set_ylabel('\nz [mm]')
ax.set_zlabel('\ny [mm]')

fig.tight_layout()
subplots_adjust(top=1, bottom = 0)

fig.set_size_inches(11.69, 8.27)
savefig(fname,dpi=300)

0.214576688407022


# Distance moyenne entre les ions Ca$^+$

In [1039]:
dist_ions = zeros(shapevar)
dist_ions_theo = zeros(shapevar)
V = zeros(shapevar)
V_theo = zeros(shapevar)
rho = zeros(shapevar)
k = 3
for k in range(elem_0):
    for j in range(elem_2):     # Udc
        rho[k][j] = 2*m*eps0/e**2*(wx[j]*2*pi)**2 # "théorique" on s'en fout
        V_theo[k][j] = N/rho[k][j]
        V[k][j] = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
    #     rho[j] = N/V_moy[j] # donné par le volume et le nombre d'ions
    #     dist_ions[j] = ( 1 / (V_moy[j]*rho[j]) ) ** (-1/3) # en µm
        dist_ions[k][j] = (V[k][j]/N)**(1/3)
        dist_ions_theo[k][j] = (V_theo[k][j]/N)**(1/3)

In [1012]:
max_HC_trj = zeros((elem_0,elem_2,3))
for j in range(elem_2):     # Udc
    print('j - k')
    for k in range(elem_0): # try
        max_HC_trj[k,j,0] = max(abs( r_x[k][my_order[j]] ))
        max_HC_trj[k,j,1] = max(abs( r_y[k][my_order[j]] ))
        max_HC_trj[k,j,2] = max(abs( r_z[k][my_order[j]] ))

j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k


In [1073]:
fig = plt.figure(num='Dist ions et GMol',clear=True)
# gs = gridspec.GridSpec(2, 2, width_ratios=[8, 4])   gs[0]
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex = ax1)
# ax3 = fig.add_subplot(gs[1])
# ax4 = fig.add_subplot(gs[3], projection='3d')

for j in range(elem_2):
    for k in range(elem_0):
        Vtmp = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
        dist_ionstmp = (Vtmp/N)**(1/3)
        ax1.plot(Udc[j],dist_ionstmp*1e6,'o',color='xkcd:azul',markerfacecolor='none')
    ax1.plot(Udc[j],mean(dist_ions[:,j])*1e6,'o',marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')
# ax1.set_xlabel(r'$U_{DC}$ [V]')
ax1.set_ylabel(r'$d_{moy}$ Ca$^+$ [µm]')
ax1.grid()

for j in range(elem_2):
    for k in range(elem_0):
        r_sortie = sqrt(max_HC_trj[k,j,0]**2 + max_HC_trj[k,j,1] **2 )
        ax2.plot(Udc[j],r_sortie*1e9,'o',color='xkcd:azul',markerfacecolor='none')
    r_sortie = sqrt(mean(max_HC_trj[:,j,0])**2 + mean(max_HC_trj[:,j,1]) **2 )
    ax2.plot(Udc[j],r_sortie*1e9,marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')

#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,1])*1e9,color='xkcd:azul',marker='+')
#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,2])*1e9,color='xkcd:brown',marker='+')
ax2.set_xlabel(r'$U_{DC}$ [V]')
ax2.set_ylabel('r sortie max GMol [nm]')
ax2.grid()

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.015)

In [733]:
# Dimensions nuage
k = 3
j = 3
for k in range(elem_0):
    print(k,dim_nu[k][j]*1e3)
print(mean(dim_nu[k][j][0])*1e3)
print(mean(dim_nu[k][j][1])*1e3)
print(mean(dim_nu[k][j][2])*1e3)

0 [ 42.99489219  42.39978708 287.85712076]
1 [ 42.65120231  42.62405271 287.80382345]
2 [ 42.76592369  42.36064041 287.9562312 ]
3 [ 42.80978618  42.9812087  287.79357738]
4 [ 43.10244486  42.58837862 287.82542209]
5 [ 42.74313454  42.65863536 287.81734027]
6 [ 42.98327599  42.79459744 287.91804182]
7 [ 42.40679459  42.44336686 288.02431727]
8 [ 42.53358384  42.95718033 288.170463  ]
9 [ 42.46642501  42.44584139 284.50894996]
10 [ 42.63868457  42.91913364 287.94984749]
11 [ 42.89074924  42.37976094 285.0333752 ]
12 [ 42.65593398  42.5559912  285.48981225]
13 [ 42.44178121  42.76641178 287.79992637]
14 [ 42.54709688  42.71390501 287.95039865]
15 [ 42.72564729  42.77394151 287.88797432]
16 [ 42.81854008  42.85170789 287.80453097]
17 [ 42.51030081  42.82443334 287.9259849 ]
18 [ 42.6552673   42.74243825 287.8751817 ]
19 [ 42.7161711   42.38702035 287.66832854]
20 [ 42.44696738  42.72147073 288.03997798]
21 [ 42.9640179   42.79730769 287.90541479]
22 [ 42.66109878  42.80042336 287.93869896

In [735]:
dist_ions*1e6

array([12.75797495, 12.87671733, 12.79735191, 12.92166931, 12.88805172,
       12.97905397, 12.89029699, 13.00893142, 12.94654777, 13.07686059,
       13.08469728, 12.88724796, 12.51270963, 13.0086879 , 13.12832375,
       13.11686357, 13.09201103])

In [736]:
dist_ions_theo * 1e6

array([14.33309411, 14.37808383, 14.42364379, 14.46978675, 14.51652592,
       14.56387488, 14.61184771, 14.6360726 , 14.66045894, 14.70972358,
       14.75965715, 14.78994505, 14.83071866, 14.86159591, 14.91363491,
       14.96641052, 15.07424603])

# Ecriture fichier
Garder les principales variables dans un fichier texte

In [898]:
float_formatter = lambda x: '{:.04}'.format(x)

In [83]:
# name=['Uend ', 'ax   ','az   ','qx   ','kappa','Udc  ','Urf  ','aax  ','aaz  ','aqx  ','awx  ','awz  ','aUdc ','aUend']
# var=[Uend, ax, az, qx, kappa, Udc, Urf, aax, aaz, aqx, awx, awz, aUdc, aUend]


f=open("simu_variables.txt","w+")
f.write('Python3.7 %s\n' % datetime.datetime.now())
f.write('Simulations collisions GMol -> Ca+ \n\n\n')

f.write('> Répertoire de travail\n')
f.write('%s\n\n' % work_rep)

f.write('> Conditions de simulation\n')
f.write(f'N_ions E_GMol\n')
f.write(f'{N} {e_GMol:>4}eV\n\n')

f.write(f'> Nombre total de simulations : {cond_number}\n')
f.write(f'{cond_two_name} : {elem_2} élément(s)\n')
f.write(f'{[ int(x) for x in cond_two]}\n')
f.write(f'{[ Udc[x] for x in my_order ]} V\n')
f.write(f'{cond_one_name} : {elem_1} élément(s)\n')
f.write(f'{[ int(x) for x in cond_one]}\n')
f.write(f'{[ int(x) for x in [Urf]]} V\n')
f.write(f'{cond_zero_name} : {elem_0} élément(s)\n')
f.write(f'{try_subdir} \n\n')

f.write(f'> X, Y, Z : Demi-axes moyens du nuage Ca+ [µm]\n')
f.write(f'> d_moy : Distance moyenne entre les ions du nuage Ca+ [µm]\n')
f.write(f' Udc  :     X      Y      Z      d_moy\n')
for j in range(elem_2):
    f.write(f'{Udc[j]:.2f}  :  {[ float(float_formatter(mean(dim_nu[:,my_order[j],x])*1000)) for x in [0,1,2]]}  {dist_ions[j]*1e6:.3f}\n')
    
# f.write(f'> Molécule géante - position max\n')
# f.write(f' Udc  :    rx[nm]      ry[nm]     rz[mm]\n')
# for j in range(elem_2):
#     f.write(f'{Udc[j]:.2f}  :  {[ float_formatter(r_x[my_order[j]]), r_y[my_order[j]], r_z[my_order[j]] ]}\n')

f.close()

NameError: name 'N' is not defined

In [125]:
my_q = [0.5,0.5+0.05/3,0.5+2*0.05/3,0.55,0.55+0.05/3,0.55+2*0.05/3,0.6,0.6+0.05/3,0.6+2*0.05/3, 0.65, 0.65+0.05/3, 0.65+0.1/3, 0.7] # [0.6, 0.625, 0.650, 0.675, 0.7]
print(my_q)
coucou = []
figure('pouet')
print('q','   ','Vrf')
for k,l in enumerate(my_q):
    coucou.append(tick_function_q(l))
    print(f'{l:2.5f}',f'{tick_function_q(l):2.2f}')
scatter(my_q,coucou)

[0.5, 0.5166666666666667, 0.5333333333333333, 0.55, 0.5666666666666668, 0.5833333333333334, 0.6, 0.6166666666666667, 0.6333333333333333, 0.65, 0.6666666666666667, 0.6833333333333333, 0.7]
q     Vrf
0.50000 53.84
0.51667 55.64
0.53333 57.43
0.55000 59.23
0.56667 61.02
0.58333 62.82
0.60000 64.61
0.61667 66.41
0.63333 68.20
0.65000 70.00
0.66667 71.79
0.68333 73.59
0.70000 75.38
